In [2]:
# first we import the packages we need

In [3]:
import pandas as pd
import numpy as np 
import sklearn as sk
from sklearn.model_selection import train_test_split


In [4]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 84.2 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install SciPy

Note: you may need to restart the kernel to use updated packages.


In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
# read our original data
# note some of those transformation is all ready made on another data file its here for show
df_all = pd.read_csv("/kaggle/input/stuff-for-work/Only_Data_With_All_Columns_Normalized_Grades.csv")

here we will do some transformations to prepare our data for the work a head 

first lets have a look on the data

In [10]:
df_all.head()

,question,model_answer,student_answer,grade,source,ID
0,how african americans were immigrated to the us,"as such, african immigrants are to be distingu...",african immigration to the united states refer...,0.0,wikiqa.csv,0
1,how african americans were immigrated to the us,"as such, african immigrants are to be distingu...",the term african in the scope of this article ...,0.0,wikiqa.csv,0
2,how african americans were immigrated to the us,"as such, african immigrants are to be distingu...",from the immigration and nationality act of 19...,0.0,wikiqa.csv,0
3,how african americans were immigrated to the us,"as such, african immigrants are to be distingu...",african immigrants in the united states come f...,0.0,wikiqa.csv,0
4,how african americans were immigrated to the us,"as such, african immigrants are to be distingu...","they include people from different national, l...",0.0,wikiqa.csv,0


In [11]:
df_all["source"].unique()

array(['wikiqa.csv', 'short_answer_grading.csv',
       'AR-ASAG-Dataset_Translated.xlsx', 'asag_dataset.csv'],
      dtype=object)

Lets see wikiqa data in farther detail

In [13]:
len(df_all[df_all["source"]=='wikiqa.csv'])

5353

In [14]:
df_all[df_all["source"]=='wikiqa.csv']["grade"].unique()

array([0., 1.])

In [15]:
len(df_all[df_all["source"]=='wikiqa.csv'][df_all["grade"]==0])

/tmp/ipykernel_34/453499894.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df_all[df_all["source"]=='wikiqa.csv'][df_all["grade"]==0])


3880

In [16]:
len(df_all[df_all["source"]=='wikiqa.csv'][df_all["grade"]==1])

/tmp/ipykernel_34/334689176.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df_all[df_all["source"]=='wikiqa.csv'][df_all["grade"]==1])


1473

Give that there are more zeros than any other grade and that there are only two gardes and the nature of the wikiqa data set we better remove it so that it do not introduce bais in the model

In [17]:
df_filtered = df_all[df_all["source"] != 'wikiqa.csv'].copy()

In [18]:
df_filtered.head()

,question,model_answer,student_answer,grade,source,ID
5353,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate the behaviour of portions of the d...,0.7,short_answer_grading.csv,0
5354,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate portions of the desired final prod...,1.0,short_answer_grading.csv,0
5355,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,a prototype program simulates the behavior of ...,0.8,short_answer_grading.csv,0
5356,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,defined in the specification phase a prototype...,1.0,short_answer_grading.csv,0
5357,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,it is used to let the users have a first idea ...,0.6,short_answer_grading.csv,0


or we import the fixed data

In [19]:
df_filtered = pd.read_csv("/kaggle/input/stuff-for-work/Only_Data_With_All_Columns_Normalized_Grades_fixed.csv")

we will have a look also here

In [20]:
df_filtered.head()

,question,model_answer,student_answer,grade,source,ID
0,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate the behaviour of portions of the d...,0.7,short_answer_grading.csv,0
1,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate portions of the desired final prod...,1.0,short_answer_grading.csv,0
2,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,a prototype program simulates the behavior of ...,0.8,short_answer_grading.csv,0
3,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,defined in the specification phase a prototype...,1.0,short_answer_grading.csv,0
4,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,it is used to let the users have a first idea ...,0.6,short_answer_grading.csv,0


In [21]:
len(df_filtered)

4722

In [22]:
df_filtered["source"].unique()

array(['short_answer_grading.csv', 'AR-ASAG-Dataset_Translated.xlsx',
       'asag_dataset.csv'], dtype=object)

In [23]:
len(df_filtered[df_filtered["source"] == "short_answer_grading.csv"])

2259

In [27]:
len(df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"])

2125

In [24]:
len(df_filtered[df_filtered["source"] == "asag_dataset.csv"])

338

In [25]:
df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["question"].nunique()

48

lets fix a small issue with translated data grades it has small decmals that can cuase some bais - we want all data to have 
similar ranges 


In [26]:
# first we read them
pt=pd.read_csv(r"/kaggle/input/the-other-stuff/PT_ASAG.csv")

In [27]:
mh=pd.read_csv(r"/kaggle/input/the-other-stuff/mohler_cleaned.csv")

In [30]:
# pt is fine
pt["grade"].unique()

array([1. , 0. , 0.5])

In [31]:
# mh is not
mh["grade"].unique()

array([0.7  , 1.   , 0.8  , 0.6  , 0.4  , 0.5  , 0.9  , 0.3  , 0.2  ,
       0.   , 0.1  , 0.525, 0.75 , 0.275, 0.575, 0.875, 0.55 , 0.45 ,
       0.925, 0.975, 0.125, 0.25 , 0.675, 0.475, 0.225, 0.35 , 0.325,
       0.95 , 0.375, 0.85 , 0.725, 0.425, 0.825, 0.65 , 0.775, 0.625])

In [34]:
# lets fix it but round it up for accpectable ranges
mh_update=mh["grade"].apply(lambda x:round(x*5*2)/2/5)

In [35]:
mh_update.unique()

array([0.7, 1. , 0.8, 0.6, 0.4, 0.5, 0.9, 0.3, 0.2, 0. , 0.1])

In [36]:
# now lets put it in its place
mh["grade"]=mh_update

In [37]:
# lets check
mh["grade"].unique()

array([0.7, 1. , 0.8, 0.6, 0.4, 0.5, 0.9, 0.3, 0.2, 0. , 0.1])

now lets check the grades of translated data set

In [38]:
df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["grade"]

2259    0.600
2260    1.000
2261    0.525
2262    0.800
2263    0.700
        ...  
4379    0.800
4380    0.500
4381    0.900
4382    0.850
4383    0.800
Name: grade, Length: 2125, dtype: float64

In [39]:
df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["grade"].unique()

array([0.6  , 1.   , 0.525, 0.8  , 0.7  , 0.75 , 0.9  , 0.725, 0.85 ,
       0.5  , 0.15 , 0.45 , 0.95 , 0.65 , 0.55 , 0.25 , 0.775, 0.35 ,
       0.2  , 0.3  , 0.4  , 0.05 , 0.   , 0.1  , 0.425, 0.575, 0.675,
       0.175, 0.275, 0.325, 0.225, 0.875, 0.625])

lets do the same for them 

In [40]:
df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["grade"].apply(lambda x:round(x*5*2)/2/5)

2259    0.6
2260    1.0
2261    0.5
2262    0.8
2263    0.7
       ... 
4379    0.8
4380    0.5
4381    0.9
4382    0.8
4383    0.8
Name: grade, Length: 2125, dtype: float64

In [41]:
update = df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["grade"].apply(lambda x:round(x*5*2)/2/5)

In [42]:
update.unique()

array([0.6, 1. , 0.5, 0.8, 0.7, 0.9, 0.2, 0.4, 0.3, 0. , 0.1])

In [43]:
df_filtered.loc[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx","grade"] = update

In [44]:
# lets check
df_filtered[df_filtered["source"] == "AR-ASAG-Dataset_Translated.xlsx"]["grade"].unique()

array([0.6, 1. , 0.5, 0.8, 0.7, 0.9, 0.2, 0.4, 0.3, 0. , 0.1])

In [46]:
# lets make sure all grades is on its acceple ranges 
df_filtered["grade"].unique()

array([0.7  , 1.   , 0.8  , 0.6  , 0.4  , 0.5  , 0.9  , 0.3  , 0.2  ,
       0.   , 0.1  , 0.825, 0.75 , 0.35 , 0.45 , 0.725, 0.65 , 0.95 ])

it is not lets check why

In [47]:
# lets see where is the problem 
df_filtered[df_filtered["grade"].isin([0.8250,0.75,0.35,0.45,0.725,0.65,0.95])]

,question,model_answer,student_answer,grade,source,ID
680,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,"log -lrb- log -lrb- a -rrb- -rrb- , a to the...",0.825,short_answer_grading.csv,0
682,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,log -lrb- log a -rrb- ; 2 to the power of -lr...,0.750,short_answer_grading.csv,0
685,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,"to the power of -lrb- log a -rrb- , log -lrb-...",0.350,short_answer_grading.csv,0
686,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,log -lrb- log a -rrb- ; a!; a to the power of...,0.450,short_answer_grading.csv,0
690,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,from lowest to longest. log -lrb- log a -rrb- ...,0.725,short_answer_grading.csv,0
696,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,to the power of -lrb- log a -rrb- a to the pow...,0.650,short_answer_grading.csv,0
698,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,log -lrb- log a -rrb- ; a to the power of 2; ...,0.825,short_answer_grading.csv,0
699,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,a to the power of 2; a!; a to the power of 3; ...,0.450,short_answer_grading.csv,0
701,order the following functions by their running...,log(log a); 2^(log a) ; a^2 ; a^3; a!,"log -lrb- long a -rrb- , 2 to the power of -l...",0.950,short_answer_grading.csv,0


In [48]:
# the lenght of the the records that have the problem 
len(df_filtered[df_filtered["grade"].isin([0.8250,0.75,0.35,0.45,0.725,0.65,0.95])])

9

so the issue was some records that is not filtered correctly for math so we will remove them because they are math and have unaccpectable ranges

In [49]:
df_filtered = df_filtered[~df_filtered["grade"].isin([0.8250,0.75,0.35,0.45,0.725,0.65,0.95])]

In [51]:
# now checking
df_filtered["grade"].unique()
# prefect

array([0.7, 1. , 0.8, 0.6, 0.4, 0.5, 0.9, 0.3, 0.2, 0. , 0.1])

In [52]:
len(df_filtered)

4713

now its fixed

now lets split them into three data frames one for each source

here we have to kind of split first normal split secand Sp split in which we make the question unique in both traning and test data set ( no simliar question in train and test data set) this for farther testing in later parts of the project

this step aim to transform our data into parts that our model consume one after another to see the if the translated data good or bad for our model ( introduce bais because the translation maybe be bad ) as well for the asag data set that we have with it some issues

In [55]:
# first og for the data we started with in the first phase
df_og = df_filtered[df_filtered["source"] == "short_answer_grading.csv"].copy()

first the Sp split then the normal split

In [56]:
df_og.head()

,question,model_answer,student_answer,grade,source,ID
0,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate the behaviour of portions of the d...,0.7,short_answer_grading.csv,0
1,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate portions of the desired final prod...,1.0,short_answer_grading.csv,0
2,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,a prototype program simulates the behavior of ...,0.8,short_answer_grading.csv,0
3,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,defined in the specification phase a prototype...,1.0,short_answer_grading.csv,0
4,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,it is used to let the users have a first idea ...,0.6,short_answer_grading.csv,0


now lets split the data into training and testing

In [57]:
# lets get the 20 percent for testing
df_og["question"].nunique()/10*2

16.6

In [58]:
ques = df_og["question"].unique()

In [59]:
import random

In [60]:
# now lets shaffle them to get the split random
random.shuffle(ques)

In [61]:
ques

array(['what are the two different ways of specifying the length of an array?',
       'which implementation (array-based is. list-based) is preferred for a queue, and why?',
       'what is the difference between a data member and a local variable inside a member function?',
       'what are the main advantages associated with object-oriented programming?',
       'what is the number of operations for insertion sort under a best-case scenario, and what is the best-case scenario?',
       'what is typically included in a class definition?',
       'how do you delete a node from a binary search tree?',
       'when defining a recursive function, what are possible causes for infinite recursion?',
       'what stages in the software life cycle are influenced by the testing stage?',
       'how are overloaded functions differentiated by the compiler?',
       'how are objects initialized when they are created?',
       'what is the main advantage associated with function arguments that are

In [62]:
# test the code
df_og[~df_og["question"].isin(ques[:15])]

,question,model_answer,student_answer,grade,source,ID
0,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate the behaviour of portions of the d...,0.7,short_answer_grading.csv,0
1,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,to simulate portions of the desired final prod...,1.0,short_answer_grading.csv,0
2,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,a prototype program simulates the behavior of ...,0.8,short_answer_grading.csv,0
3,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,defined in the specification phase a prototype...,1.0,short_answer_grading.csv,0
4,what is the role of a prototype program in pro...,to simulate the behaviour of portions of the d...,it is used to let the users have a first idea ...,0.6,short_answer_grading.csv,0
...,...,...,...,...,...,...
2254,what data structure is more appropriate for sc...,queue,a queue. it would not be unfair for the firs...,1.0,short_answer_grading.csv,0
2255,what data structure is more appropriate for sc...,queue,"a queue, because it takes the job that has wai...",1.0,short_answer_grading.csv,0
2256,what data structure is more appropriate for sc...,queue,"a queue is more appropriate, because the clien...",1.0,short_answer_grading.csv,0
2257,what data structure is more appropriate for sc...,queue,i would say that a queue is better because the...,1.0,short_answer_grading.csv,0


In [63]:
# now lets assign our data
val_df_og_sp2=df_og[df_og["question"].isin(ques[:15])].copy()

In [64]:
train_df_og_sp2=df_og[~df_og["question"].isin(ques[:15])].copy()

we will do this for every source as we go and feed them to the data one at time to test if the accuracy increases

now the normal split

In [65]:
train_df_og, val_df_og = train_test_split(df_og, test_size=0.2, random_state=42)

In [66]:
len(val_df_og)

450

In [64]:
len(train_df_og)

1800

now we will do the same for other data sets

In [67]:
df_translated = df_filtered[df_filtered["source"] == 'AR-ASAG-Dataset_Translated.xlsx'].copy()

In [68]:
df_translated["question"].nunique()

48

In [69]:
df_translated["question"].nunique()/10*2


9.6

In [70]:
ques_t = df_translated["question"].unique()

In [71]:
random.shuffle(ques_t)

In [72]:
ques_t

array(['why is it difficult to use regular legal legislation to combat cybercrime?',
       'what are the consequences of infecting an information system with a trojan horse?',
       'what are the consequences of using biometric techniques to access the information system?',
       'what type of crimes does wikileaks commit and where does it rank in terms of the general classification?',
       'mention the physical characteristics that are commonly used in biometry',
       'what is the difference between a virus and a download program?',
       'what are the pillars of securing information against potential violations by users of the information system?',
       'mention the types of financial cyber crimes',
       'how is the crime of industrial espionage carried out on the internet?',
       'explain why organizations often do not think about supporting a security policy early in the design and development of the information system?',
       'what are the main types of malicious s

In [73]:
df_translated[df_translated["question"].isin(ques_t[:8])]

,question,model_answer,student_answer,grade,source,ID
2696,what are the pillars of securing information a...,"identifying the user, proving the user's ident...",get a good security system.,0.0,AR-ASAG-Dataset_Translated.xlsx,1
2697,what are the pillars of securing information a...,"identifying the user, proving the user's ident...","determining the user’s identity, establishing ...",1.0,AR-ASAG-Dataset_Translated.xlsx,1
2698,what are the pillars of securing information a...,"identifying the user, proving the user's ident...","availability: in this case, the information mu...",0.2,AR-ASAG-Dataset_Translated.xlsx,1
2699,what are the pillars of securing information a...,"identifying the user, proving the user's ident...","using the login system (username, password), u...",0.0,AR-ASAG-Dataset_Translated.xlsx,1
2700,what are the pillars of securing information a...,"identifying the user, proving the user's ident...",confidentiality - verifying identity - confirm...,0.4,AR-ASAG-Dataset_Translated.xlsx,1
...,...,...,...,...,...,...
4379,what is the difference between a virus and a d...,the virus and the download program are cunning...,a virus is a malicious program that attaches i...,0.8,AR-ASAG-Dataset_Translated.xlsx,1
4380,what is the difference between a virus and a d...,the virus and the download program are cunning...,a virus is a program that is coupled with anot...,0.5,AR-ASAG-Dataset_Translated.xlsx,1
4381,what is the difference between a virus and a d...,the virus and the download program are cunning...,the virus is a program hidden inside another p...,0.9,AR-ASAG-Dataset_Translated.xlsx,1
4382,what is the difference between a virus and a d...,the virus and the download program are cunning...,the virus is associated with another program a...,0.8,AR-ASAG-Dataset_Translated.xlsx,1


In [74]:
val_df_translated_sp2=df_translated[df_translated["question"].isin(ques_t[:8])].copy()

In [75]:
train_df_translated_sp2=df_translated[~df_translated["question"].isin(ques_t[:8])].copy()

In [76]:
train_df_translated, val_df_translated = train_test_split(df_translated, test_size=0.2, random_state=42)

In [77]:
len(df_translated)

2125

In [78]:
len(train_df_translated)

1700

In [79]:
len(val_df_translated)

425

In [80]:
df_asag = df_filtered[df_filtered["source"] == 'asag_dataset.csv'].copy()

In [81]:
df_asag["question"].nunique()/10*2

1.8

In [82]:
ques_as=df_asag["question"].unique()

In [83]:
random.shuffle(ques_as)

In [84]:
train_df_asag, val_df_asag = train_test_split(df_asag, test_size=0.2, random_state=42)

In [85]:
len(df_asag)

338

In [86]:
len(train_df_asag)

270

In [87]:
len(val_df_asag)

68

now lets start downloading our pretrained model

In [89]:
# note we use all-minilm because its light for download and faster and still give one of the best result
Model_mpnet = SentenceTransformer('all-MiniLM-L6-v2')

now lets try the model


In [90]:
from sentence_transformers import  util


In [91]:
# Single list of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

#Compute embeddings
embeddings = Model_mpnet.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [92]:
pairs

[{'index': [0, 1], 'score': tensor(0.0363, device='cuda:0')},
 {'index': [0, 2], 'score': tensor(0.0081, device='cuda:0')},
 {'index': [0, 3], 'score': tensor(-0.0247, device='cuda:0')},
 {'index': [0, 4], 'score': tensor(0.6788, device='cuda:0')},
 {'index': [0, 5], 'score': tensor(0.1310, device='cuda:0')},
 {'index': [0, 6], 'score': tensor(-0.0029, device='cuda:0')},
 {'index': [0, 7], 'score': tensor(0.0254, device='cuda:0')},
 {'index': [1, 2], 'score': tensor(-0.0368, device='cuda:0')},
 {'index': [1, 3], 'score': tensor(0.0093, device='cuda:0')},
 {'index': [1, 4], 'score': tensor(0.2105, device='cuda:0')},
 {'index': [1, 5], 'score': tensor(-0.0327, device='cuda:0')},
 {'index': [1, 6], 'score': tensor(-0.0136, device='cuda:0')},
 {'index': [1, 7], 'score': tensor(0.0116, device='cuda:0')},
 {'index': [2, 3], 'score': tensor(0.2440, device='cuda:0')},
 {'index': [2, 4], 'score': tensor(0.0230, device='cuda:0')},
 {'index': [2, 5], 'score': tensor(0.0359, device='cuda:0')},
 {'

now lets see the accuracy before we train the data

first we combine the evaluation data sets

In [94]:
# lets combine some data to test with -note this is not the finial val data
df_val= pd.concat([val_df_asag,val_df_translated,val_df_og],ignore_index=True)

In [95]:
df_val.head()

,question,model_answer,student_answer,grade,source,ID
0,explain classification and regression; what is...,classification is a task of mapping data to di...,classification is type of problem where algori...,0.5,asag_dataset.csv,2
1,describe: the structure on an can.,convolutional neural network consists of many ...,a can consists of one or more convolution laye...,0.5,asag_dataset.csv,2
2,what role does the method of steepest decent ...,steepest descent is used to update the weights...,steepest decent method helps in making the adj...,0.5,asag_dataset.csv,2
3,explain classification and regression; what is...,classification is a task of mapping data to di...,"classification: in classification, the output ...",0.5,asag_dataset.csv,2
4,"explain back propagation, use the correct tech...",backpropagation lowers the error of a mlp leve...,* back propagation is a steepest decent method...,1.0,asag_dataset.csv,2


In [96]:
len(df_val)

943

In [97]:
students_embed=Model_mpnet.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [98]:
students_embed[0:5]

tensor([[-0.0946, -0.0156, -0.0761,  ...,  0.0455,  0.0861, -0.0078],
        [-0.0234, -0.0341,  0.0100,  ...,  0.0988,  0.0271,  0.0034],
        [-0.0823, -0.0311,  0.0860,  ...,  0.0779, -0.0327, -0.0545],
        [-0.1074, -0.0064, -0.0672,  ...,  0.0440,  0.1046,  0.0046],
        [-0.0284, -0.0810, -0.0003,  ...,  0.1455,  0.0366, -0.0260]],
       device='cuda:0')

In [99]:
len(students_embed)

943

In [100]:
model_answer_embed=Model_mpnet.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [101]:
model_answer_embed[0:5]

tensor([[-0.0798, -0.0142, -0.0557,  ...,  0.0513,  0.0685, -0.0437],
        [-0.0476, -0.0983,  0.0570,  ...,  0.1281,  0.0094, -0.0085],
        [-0.0659,  0.0071,  0.0173,  ...,  0.0338, -0.0074, -0.0492],
        [-0.0798, -0.0142, -0.0557,  ...,  0.0513,  0.0685, -0.0437],
        [-0.0547, -0.0742,  0.0974,  ...,  0.1094,  0.0209, -0.0786]],
       device='cuda:0')

In [102]:
len(model_answer_embed)

943

nice all things working fine

we need first a function that takes numbers form 0 to 1 (out put of our model)  and outputs numbers form a 0 to 5 ( the range we assume the prof want)

we will modifiy this code farther to contain any range (that the prof want) we want this version is for testing 

In [104]:
import math

In [105]:
def Gradanizer(num, grade_mode="Fair"):
    """
    This function take the garde form 0 to 1 and output that garde in form 0 to 5

    parameters
    num: the garde in form 0 to 1
    grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
    common, lose_ends: more skewed into higher grades and lower grades]
    """
    def get_region_value_f(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
        (4.0909, 4.5455), (4.5455, 6)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
        
    def get_region_value_e(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
    def get_region_value_l(number):    
        intervals = [
        (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
        (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
        (3.1429, 3.5000)]
        values=[0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]

    if (grade_mode == "fair"):
        num_f=num*5
        return get_region_value_f(num_f)
    elif (grade_mode == "easy"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        else:
            num_e = num*5
            return get_region_value_e(num_e)
    elif (grade_mode == "lose_ends"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        elif (0.2>=num):
            return 0
        else:
            num_l = num*5
            return get_region_value_l(num_l)
    else:
        print("not a valid mode ")


            

            
        


now lets test it


In [107]:
Gradanizer(.85,"easy")

5

In [108]:
Gradanizer(-0.85,"fair")

0

In [109]:
Gradanizer(.18,"lose_end")

not a valid mode 


In [110]:
Gradanizer(.18,"lose_ends")

0

now that Gradanizer is test and good to go lets make the evaluation function

NOTE VERY IMPORTANT NOTE FOR THIS FUNCTION YOU MUSY INSTALL SENTENCE TRANSFORMERS AND IMPORT util FORM IT 

pip install -U sentence-transformers

AND NOTE YOU NEED THE Gradanzier FUNCTION TO RUN AS WELL BEFORE YOU RUN THIS FUNCTION

and from scipy.stats import spearmanr


from sklearn.metrics import r2_score

In [111]:
# VERY IMPORTANT TO INSTALL SENTENCE TRANSFORMATERS
# pip install -U sentence-transformers
from sentence_transformers import util


In [112]:
from scipy.stats import spearmanr
from sklearn.metrics import r2_score

In [113]:
def Evaluation(embeddings_1=None, embeddings_2=None,test_data=None,mode="easy"):
    """
    this function takes two lists of embeddings in the form of torch tensor or list or df column one for the student answer and other for model answer
    or any two sentences if this function used in any other code and the test data frame and the mode of Gradazier and output MAE % and r_squared and 
    correlation_coefficient

    parameters
    embeddings_1:list or tensor or df column that hold the sentence embeddings (student answer embeddings)
    embeddings_2:list or tensor or df column that hold the sentence embeddings (model answer embeddings)
    data: the test data frame
    mode: the mode of which gradazier is used ["fair", "easy", "lose_ends"]
    """
    from sentence_transformers import util
    if (len(embeddings_1) != len(embeddings_2)):
        print("embeddings_1 and embeddings_2 are not the same length")
        return None
    elif (mode not in ["easy","lose_ends","fair"]):
        print("not valid mode")
        return None
    else:
        predicted = []
        for i in range(len(embeddings_1)):
            predicted.append(Gradanizer(util.cos_sim(embeddings_1[i], embeddings_2[i]),mode))
        transformed_grade=[]
        for i in range(len(embeddings_1)):
            #transformed_grade.append(Gradanizer(test_data["grade"][i],mode))
            transformed_grade.append(test_data["grade"][i]*5)
        arr_predicted =np.array(predicted)
        arr_grade=np.array(transformed_grade)
        MAE = (1-((np.sum(np.abs(arr_predicted-arr_grade))/len(embeddings_1))/5))*100
        correlation_coefficient, p_value = spearmanr(arr_grade, arr_predicted)
        r_squared = r2_score(arr_grade, arr_predicted)
        dif =arr_predicted-arr_grade
        hits_precentage= (np.count_nonzero(dif == 0)/len(embeddings_1))
        #return MAE,correlation_coefficient,r_squared
        #if you want the hit precentage tag the above and untag the blew
        return MAE,correlation_coefficient,hits_precentage
    

In [108]:
df_val["grade"].unique()

array([0.5, 1. , 0. , 0.8, 0.3, 0.6, 0.1, 0.7, 0.2, 0.4, 0.9])

In [114]:
precentage,correlation_coefficient,hit_precentage = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [115]:
print(precentage,correlation_coefficient,hit_precentage)

76.24602332979852 0.36774975219946904 0.13149522799575822


form we can see its accuracy is 76.2 which is good but correlated and hit_precentage can be better

Now let's fine tune the model with our data and see the difference

we feed the data into three steps one for each source and see if the accuracy increases

First of all we make a data loader fuction to input our data into the model

we need those in this from sentence_transformers import SentenceTransformer, InputExample, losses


In [116]:
from sentence_transformers import InputExample, losses

In [117]:
# this is a local data loader to load our data into the sentenace transformers model
def data_loader_local(data=None):
    input_data = []
    for index, row in data.iterrows():
        student_answer = row['student_answer']
        model_answer = row['model_answer']
        grade = row['grade']
    
        # Create InputExample instances
        input_data.append(InputExample(texts=[student_answer, model_answer], label=grade))
    return(input_data)

In [124]:
from torch.utils.data import DataLoader, Dataset

In [125]:
train_dataset_og = data_loader_local(train_df_og)


In [126]:
len(train_dataset_og)

1800

now lets load the data and make the lose function

we need from torch.utils.data import DataLoader for this

In [127]:
from torch.utils.data import DataLoader

In [128]:
train_dataloader_og = DataLoader(train_dataset_og, shuffle=True, batch_size=16)

note here we use ContrastiveLoss for testing

In [129]:
train_loss_og = losses.ContrastiveLoss(model=Model_mpnet)

note Warmup steps best practice is 10% of the training data so 0.1 *1807 =180

In [130]:
# lets train the first data set
Model_mpnet.fit(train_objectives=[(train_dataloader_og, train_loss_og)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

now lets try it

In [131]:
students_embed=Model_mpnet.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [132]:
model_answer_embed=Model_mpnet.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [133]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [134]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

78.3881230116649 0.6347460615746788 0.29056203605514314


so its getting better so we confirmed - which was obvious -  that training give good feed back

now lets for our secand test which is which is the better loss function form my studies it would be cosine loss function but lets comfirm as we tried Contrastive lose form above we will procced with the same every thing but different loss function

now we will try different loss function

In [135]:
Model_mpnet_l2 = SentenceTransformer('all-MiniLM-L6-v2')

In [136]:
students_embed=Model_mpnet_l2.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [137]:
model_answer_embed=Model_mpnet_l2.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [138]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [139]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

76.24602332979852 0.36774975219946904 0.13149522799575822


In [141]:
train_loss_og_l2 = losses.OnlineContrastiveLoss(model=Model_mpnet_l2)

In [142]:
Model_mpnet_l2.fit(train_objectives=[(train_dataloader_og, train_loss_og_l2)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

In [143]:
students_embed=Model_mpnet_l2.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [144]:
model_answer_embed=Model_mpnet_l2.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [145]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [146]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

70.75291622481443 0.09356686340766403 0.30965005302226933


form this we concould this is soo bad skip to the last loss function

In [147]:
Model_mpnet_l3 = SentenceTransformer('all-MiniLM-L6-v2')

In [148]:
students_embed=Model_mpnet_l3.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [149]:
model_answer_embed=Model_mpnet_l3.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [150]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [151]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

76.24602332979852 0.36774975219946904 0.13149522799575822


In [152]:
train_loss_og_l3 = losses.CosineSimilarityLoss(model=Model_mpnet_l3)

In [153]:
Model_mpnet_l3.fit(train_objectives=[(train_dataloader_og, train_loss_og_l3)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

In [154]:
students_embed=Model_mpnet_l3.encode(df_val["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [155]:
model_answer_embed=Model_mpnet_l3.encode(df_val["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [156]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [157]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

83.23435843054084 0.6648220073063472 0.2449628844114528


better accurcy but worst hit precentage it will be considered as an option

now lets see if modeling seeing the question (question is not unique in test data set) make a noticable difference 

In [158]:
Model_mpnet_sp2 = SentenceTransformer('all-MiniLM-L6-v2')

In [159]:
train_dataset_og_sp2 = data_loader_local(train_df_og_sp2)

In [160]:
train_dataloader_og_sp2 = DataLoader(train_dataset_og_sp2, shuffle=True, batch_size=16)

In [161]:
len(train_dataloader_og_sp2)

114

In [162]:
len(train_dataset_og)

1800

In [163]:
train_loss_og_sp2 = losses.ContrastiveLoss(model=Model_mpnet_sp2)

In [164]:
Model_mpnet_sp2.fit(train_objectives=[(train_dataloader_og_sp2, train_loss_og_sp2)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

In [165]:
students_embed=Model_mpnet_sp2.encode(val_df_og_sp2["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [166]:
model_answer_embed=Model_mpnet_sp2.encode(val_df_og_sp2["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [167]:
# we reset index because it was differecnt in some version and the old index some gaps so we do that to aviod the error that accured and we are
# fixing it form the future
val_df_og_sp2_rested =val_df_og_sp2.reset_index(drop=True)

In [168]:
val_df_og_sp2.head()

,question,model_answer,student_answer,grade,source,ID
28,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,refining and possibly the design if the testin...,0.7,short_answer_grading.csv,0
29,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,the implementation phase and the maintenance p...,0.8,short_answer_grading.csv,0
30,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,"verification, coding, refining the solution an...",0.9,short_answer_grading.csv,0
31,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,in rup the stages in the software life cycle a...,0.6,short_answer_grading.csv,0
32,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,"refining the solution, production and maintena...",0.6,short_answer_grading.csv,0


In [169]:
val_df_og_sp2_rested.head()

,question,model_answer,student_answer,grade,source,ID
0,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,refining and possibly the design if the testin...,0.7,short_answer_grading.csv,0
1,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,the implementation phase and the maintenance p...,0.8,short_answer_grading.csv,0
2,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,"verification, coding, refining the solution an...",0.9,short_answer_grading.csv,0
3,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,in rup the stages in the software life cycle a...,0.6,short_answer_grading.csv,0
4,what stages in the software life cycle are inf...,the testing stage can influence both the codin...,"refining the solution, production and maintena...",0.6,short_answer_grading.csv,0


In [170]:
val_df_og_sp2_rested["grade"][2]

0.9

In [171]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,val_df_og_sp2_rested,"fair")

In [172]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

82.95871559633028 0.5103864205360927 0.30504587155963303


now withour being unique in test data set

In [173]:
Model_mpnet_notsp2 = SentenceTransformer('all-MiniLM-L6-v2')

In [174]:
train_dataset_og = data_loader_local(train_df_og)

In [175]:
train_dataloader_og = DataLoader(train_dataset_og, shuffle=True, batch_size=16)

In [176]:
len(train_dataloader_og)

113

In [177]:
len(train_dataset_og)

1800

In [178]:
train_loss_og_notsp2 = losses.ContrastiveLoss(model=Model_mpnet_notsp2)

In [179]:
Model_mpnet_notsp2.fit(train_objectives=[(train_dataloader_og, train_loss_og_notsp2)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

In [180]:
students_embed=Model_mpnet_notsp2.encode(val_df_og["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

In [181]:
model_answer_embed=Model_mpnet_notsp2.encode(val_df_og["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

In [182]:
# the same reseting
val_df_og_rested = val_df_og.reset_index(drop=True)

In [184]:
# checing
val_df_og_rested["grade"][0]

0.7

In [185]:
val_df_og_rested.head()

,question,model_answer,student_answer,grade,source,ID
0,when does a++ create a default constructor?,"if no constructor is provided, the compiler pr...",during compilation,0.7,short_answer_grading.csv,0
1,what is a recursive function?,a function that calls itself.,a function that calls itself. with each func...,1.0,short_answer_grading.csv,0
2,what is the main difference between a string o...,the char[] will automatically add a null \0 ch...,char array needs an end character signature an...,1.0,short_answer_grading.csv,0
3,what is the height of a tree?,the length of the longest path from the root t...,the connections between elements are called br...,0.3,short_answer_grading.csv,0
4,what does the size of operator return?,the size in bytes of its operand.,the byte size of the date stored ina variable.,0.9,short_answer_grading.csv,0


In [186]:
val_df_og.head()

,question,model_answer,student_answer,grade,source,ID
949,when does a++ create a default constructor?,"if no constructor is provided, the compiler pr...",during compilation,0.7,short_answer_grading.csv,0
482,what is a recursive function?,a function that calls itself.,a function that calls itself. with each func...,1.0,short_answer_grading.csv,0
581,what is the main difference between a string o...,the char[] will automatically add a null \0 ch...,char array needs an end character signature an...,1.0,short_answer_grading.csv,0
247,what is the height of a tree?,the length of the longest path from the root t...,the connections between elements are called br...,0.3,short_answer_grading.csv,0
1668,what does the size of operator return?,the size in bytes of its operand.,the byte size of the date stored ina variable.,0.9,short_answer_grading.csv,0


In [187]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,val_df_og_rested,"fair")

In [188]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

86.55555555555556 0.5372113002332352 0.4533333333333333


so yes it does  matter it increase correlation and hit_precentage again that was expected but we must make sure,

now lets see if removing stop words will increase the correlation

In [206]:
# a local function to remove stop words
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [198]:
# some imports we need
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [207]:
# to avoid the same problem
train_df_og_rested=train_df_og.reset_index(drop=True)

In [208]:
train_df_og_rested["model_answer"][100]

'the elements in an array can be accessed directly (as opposed to linked lists, which require iterative traversal).'

In [209]:
train_df_og["model_answer"][200]

'a collection of nodes, which has a special node called root, and the rest of the nodes are partitioned into one or more disjoint sets, each set being a tree.'

In [210]:
# testing
remove_stopwords(train_df_og_rested["model_answer"][200])
# nice working

'four ways : constant pointer constant data , constant pointer constant data , constant pointer constant data , constant pointer constant data .'

In [214]:
# now lets create a function to loop over it {the data farm}
def create_stopword_rows(df):
    df_reseted=df.reset_index(drop=True)
    lolo = []
    for i in range(len(df)):
        lolo.append(remove_stopwords(df_reseted[i]))
    return lolo
    

In [212]:
kolo =create_stopword_rows(train_df_og["model_answer"])

In [213]:
kolo[:10]

['function calls .',
 'involve repetition ; termination tests ; occur infinitely .',
 'data structure store elements , property last item added last removed ( first-in-first-out ) .',
 'address location memory .',
 'store class interface , including data members member function prototypes .',
 'data members accessed member functions inside class definition . local variables accessed inside member function defines .',
 'using constructors .',
 'makes copy function code every place function call made .',
 'data structure stores elements following first first principle . main operations queue queue de queue .',
 'data structure store elements , property last item added last removed ( first-in-first-out ) .']

In [215]:
train_df_og_rested.head()

,question,model_answer,student_answer,grade,source,ID
0,what is a recursive function?,a function that calls itself.,it is a function that calls itself. it runs ...,1.0,short_answer_grading.csv,0
1,what are the similarities between iteration an...,they both involve repetition; they both have t...,"both based on control statement, involve repet...",1.0,short_answer_grading.csv,0
2,what is a queue?,"a data structure that can store elements, whic...",is a particular kind of collection in which th...,1.0,short_answer_grading.csv,0
3,what is a pointer?,the address of a location in memory.,a pointer is a variable that holds the address...,1.0,short_answer_grading.csv,0
4,what is the role of a header-file?,"to store a class interface, including data mem...",to separate the source code from the main func...,0.7,short_answer_grading.csv,0


In [216]:
train_df_og.head()

,question,model_answer,student_answer,grade,source,ID
478,what is a recursive function?,a function that calls itself.,it is a function that calls itself. it runs ...,1.0,short_answer_grading.csv,0
1256,what are the similarities between iteration an...,they both involve repetition; they both have t...,"both based on control statement, involve repet...",1.0,short_answer_grading.csv,0
2109,what is a queue?,"a data structure that can store elements, whic...",is a particular kind of collection in which th...,1.0,short_answer_grading.csv,0
629,what is a pointer?,the address of a location in memory.,a pointer is a variable that holds the address...,1.0,short_answer_grading.csv,0
1049,what is the role of a header-file?,"to store a class interface, including data mem...",to separate the source code from the main func...,0.7,short_answer_grading.csv,0


now lets copy it and test the stop word removed data frame

In [218]:
train_df_og_stop = train_df_og_rested.copy()

In [219]:
train_df_og_stop.head()

,question,model_answer,student_answer,grade,source,ID
0,what is a recursive function?,a function that calls itself.,it is a function that calls itself. it runs ...,1.0,short_answer_grading.csv,0
1,what are the similarities between iteration an...,they both involve repetition; they both have t...,"both based on control statement, involve repet...",1.0,short_answer_grading.csv,0
2,what is a queue?,"a data structure that can store elements, whic...",is a particular kind of collection in which th...,1.0,short_answer_grading.csv,0
3,what is a pointer?,the address of a location in memory.,a pointer is a variable that holds the address...,1.0,short_answer_grading.csv,0
4,what is the role of a header-file?,"to store a class interface, including data mem...",to separate the source code from the main func...,0.7,short_answer_grading.csv,0


In [220]:
train_df_og_stop["model_answer"] = create_stopword_rows(train_df_og_stop["model_answer"])

In [221]:
train_df_og_stop["student_answer"] = create_stopword_rows(train_df_og_stop["student_answer"])

In [222]:
train_df_og_stop.head()

,question,model_answer,student_answer,grade,source,ID
0,what is a recursive function?,function calls .,function calls . runs multiple times base case...,1.0,short_answer_grading.csv,0
1,what are the similarities between iteration an...,involve repetition ; termination tests ; occur...,"based control statement , involve repetition ,...",1.0,short_answer_grading.csv,0
2,what is a queue?,"data structure store elements , property last ...",particular kind collection entities collection...,1.0,short_answer_grading.csv,0
3,what is a pointer?,address location memory .,pointer variable holds address given variable ...,1.0,short_answer_grading.csv,0
4,what is the role of a header-file?,"store class interface , including data members...","separate source code main function , allows me...",0.7,short_answer_grading.csv,0


In [223]:
Model_mpnet_stopword = SentenceTransformer('all-MiniLM-L6-v2')

In [224]:
train_dataset_sw = data_loader_local(train_df_og_stop)

In [225]:
train_dataloader_sw = DataLoader(train_dataset_sw, shuffle=True, batch_size=16)

In [226]:
train_loss_sw = losses.ContrastiveLoss(model=Model_mpnet_stopword)

In [227]:
Model_mpnet_stopword.fit(train_objectives=[(train_dataloader_sw, train_loss_sw)], epochs=5, warmup_steps=180)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

In [228]:
students_embed=Model_mpnet_stopword.encode(create_stopword_rows(df_val["student_answer"]),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [229]:
model_answer_embed=Model_mpnet_stopword.encode(create_stopword_rows(df_val["model_answer"]),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [230]:
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val,"fair")

In [231]:
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

77.85790031813362 0.6353907219900337 0.28525980911983034


its has ever so slightly negative effect so we will not bather do it note - we know that we should not remove them but we wanted to make sure so we tried-.

now lets get the data together and start modeling using our insights

In [232]:
mh_traing, mh_test =train_test_split(mh, test_size=0.2, random_state=42)

In [237]:
# note here we removed the below line because it is bad for our model is bad four model i will spoil it for you
# we still put it for you to not get confused why we ingnore it
#pt_traing, pt_test =train_test_split(pt_nomath, test_size=0.2, random_state=42)

In [238]:
df_val_final = pd.concat([df_val,mh_test],ignore_index=True)

In [240]:
# this val is to test if the translated data is bad
df_val_final_no3 = pd.concat([val_df_asag,mh_test,val_df_og],ignore_index=True)

In [241]:
df_val_final.head()

,question,model_answer,student_answer,grade,source,ID
0,explain classification and regression; what is...,classification is a task of mapping data to di...,classification is type of problem where algori...,0.5,asag_dataset.csv,2
1,describe: the structure on an can.,convolutional neural network consists of many ...,a can consists of one or more convolution laye...,0.5,asag_dataset.csv,2
2,what role does the method of steepest decent ...,steepest descent is used to update the weights...,steepest decent method helps in making the adj...,0.5,asag_dataset.csv,2
3,explain classification and regression; what is...,classification is a task of mapping data to di...,"classification: in classification, the output ...",0.5,asag_dataset.csv,2
4,"explain back propagation, use the correct tech...",backpropagation lowers the error of a mlp leve...,* back propagation is a steepest decent method...,1.0,asag_dataset.csv,2


In [242]:
len(df_val_final)

1377

In [243]:
train_og_final = pd.concat([mh_traing, train_df_og],ignore_index=True)

In [244]:
train_og_final.head()

,question,model_answer,student_answer,grade,source,ID
0,how are linked lists passed as arguments to a ...,by reference.,linked lists permit insertion and removal of n...,0.3,mohler_dataset_edited.csv,4
1,what is the role of a header-file?,"to store a class interface, including data mem...",it contains reusable source code for use by ot...,0.6,mohler_dataset_edited.csv,4
2,"briefly, how does selection sort work?",it selects the minimum from an array and place...,you select the smallest element and place it o...,1.0,mohler_dataset_edited.csv,4
3,what are the access-specifiers that can be use...,private and public.,"local, global, local-variable, function specific",0.4,mohler_dataset_edited.csv,4
4,what is the number of operations for insertion...,n (the length of the array) operations achieve...,"n operations, where n is the number of items.",0.8,mohler_dataset_edited.csv,4


In [245]:
len(train_og_final)

3534

In [246]:
def remove_math_questions(df):
  
  # Combine keywords and regex for math identification
    math_keywords = ["equation", "calculate", "geometry", "algebra"]
    math_regex = r"[\+\-\*\/\$\^#@]"

  # Identify rows containing math using keywords or expressions in question OR answer
    math_questions = df[
      (df["question"].str.contains("|".join(math_keywords), na=False)) |
      (df["question"].str.contains(math_regex, na=False)) |
      (df["model_answer"].str.contains(math_regex, na=False))
  ]


  # Drop the rows containing math questions
    df_without_math = df.drop(math_questions.index)

    return df_without_math

In [247]:
# same we removed pt the the reason below
#pt_nomath =remove_math_questions(pt)

In [248]:
#len(pt_nomath)

In [249]:
train_translated_final = train_df_translated.copy()

In [250]:
train_translated_final.head()

,question,model_answer,student_answer,grade,source,ID
2451,define the term money laundering,legitimizing dirty money often derived from il...,changing records and data and returning money ...,0.8,AR-ASAG-Dataset_Translated.xlsx,1
3491,what are the consequences of using biometric t...,"security and privacy, the alternative to carry...","the consequences are: security and privacy, su...",0.8,AR-ASAG-Dataset_Translated.xlsx,1
2739,how is the strength of the security chain of a...,it is measured by the strength of its weakest ...,it is measured by its weakest link,0.5,AR-ASAG-Dataset_Translated.xlsx,1
3369,what aspects should be enriched in traditional...,developing the technical expertise of investig...,"protecting the human, psychological and indust...",0.2,AR-ASAG-Dataset_Translated.xlsx,1
3561,what are the consequences of applying a penetr...,reports to the organization on how the system ...,"good, ethical and legal purposes.",0.0,AR-ASAG-Dataset_Translated.xlsx,1


In [251]:
train_translated_final["grade"].value_counts()

grade
0.8    337
0.6    287
0.4    203
1.0    194
0.2    168
0.7    109
0.5    108
0.9     99
0.0     86
0.3     69
0.1     40
Name: count, dtype: int64

In [252]:
pt_2 =pd.read_csv(r"/kaggle/input/the-other-stuff/PT_ASAG.csv")

In [253]:
pt_2["grade"].value_counts()

grade
0.0    4269
0.5    1740
1.0    1107
Name: count, dtype: int64

In [229]:
#pt_nomath["grade"].value_counts()

In [254]:
pt["grade"].value_counts()

grade
0.0    4269
0.5    1740
1.0    1107
Name: count, dtype: int64

so the reason why we remove pt data set is because its grade distribution is so much different form our other data distribution which is so bad and will introduce bais into our model (I tried)

In [255]:
mh["grade"].value_counts()

grade
1.0    1040
0.9     284
0.8     229
0.7     172
0.6     162
0.5     112
0.4     104
0.3      39
0.2      15
0.0       8
0.1       3
Name: count, dtype: int64

In [256]:
len(train_translated_final)

1700

In [257]:
train_translated_final["grade"].value_counts()

grade
0.8    337
0.6    287
0.4    203
1.0    194
0.2    168
0.7    109
0.5    108
0.9     99
0.0     86
0.3     69
0.1     40
Name: count, dtype: int64

nice distribution

lets now train our model

In [258]:
# note here you can change the model by change the model name 'all-distilroberta-v1' but we found it to be the best
final_model = SentenceTransformer('all-distilroberta-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [259]:
train_dataset_og = data_loader_local(train_og_final)

In [260]:
len(train_dataset_og)

3534

In [261]:
train_dataloader_og = DataLoader(train_dataset_og, shuffle=True, batch_size=16)

In [262]:
train_loss_og = losses.CosineSimilarityLoss(model=final_model)

In [263]:
final_model.fit(train_objectives=[(train_dataloader_og, train_loss_og)], epochs=5, warmup_steps=330)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

note form here onword for all data sets will be hashtaged as after this test we discovered that translated data in most cases introduce
bais and so we removed form val data in withour translated which will appear and we provide the result form with all in comments for review

note we found easy to be the best mode so we will contiune with it

In [268]:
#for all
#students_embed=final_model.encode(df_val_final["student_answer"].to_list(),convert_to_tensor=True)

In [269]:
# with no translated
students_embed=final_model.encode(df_val_final_no3["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [270]:
#for all
#model_answer_embed=final_model.encode(df_val_final["model_answer"].to_list(),convert_to_tensor=True)

In [271]:
# with no translated
model_answer_embed=final_model.encode(df_val_final_no3["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [272]:
# for all
#precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val_final,"easy")

In [273]:
# with no translated
precentage_1,correlation_coefficient_1,hit_precentage_1 = Evaluation(students_embed,model_answer_embed,df_val_final_no3,"easy")

In [274]:
#for all
#print(precentage_1,correlation_coefficient_1,hit_precentage_1)

 it was 85.63543936092955 | 0.6749062070572189 | 0.4567901234567901


In [275]:
# with no translation
print(precentage_1,correlation_coefficient_1,hit_precentage_1)

92.6155462184874 0.7903038917082691 0.5724789915966386


Now we know its accurcy is getting better so lets to improve it farther 

lets train it on asag

In [276]:
train_dataset_asag = data_loader_local(train_df_asag)

In [277]:
len(train_dataset_asag)

270

In [278]:
train_dataloader_asag = DataLoader(train_dataset_asag, shuffle=True, batch_size=16)

In [279]:
train_loss_asag = losses.CosineSimilarityLoss(model=final_model)

In [280]:
final_model.fit(train_objectives=[(train_dataloader_asag, train_loss_asag)], epochs=1, warmup_steps=27)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

now lets test it

In [306]:
# with not tran
students_embed_asag=final_model.encode(df_val_final_no3["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [307]:
# for all
#students_embed_asag=final_model.encode(df_val_final["student_answer"].to_list(),convert_to_tensor=True)

In [308]:
# without trans
model_answer_embed_asag=final_model.encode(df_val_final_no3["model_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [284]:
# with all 
#model_answer_embed_asag=final_model.encode(df_val_final["model_answer"].to_list(),convert_to_tensor=True)

In [316]:
# with out trans
precentage_2,correlation_coefficient_2,hit_precentage_2 = Evaluation(students_embed_asag,model_answer_embed_asag,df_val_final_no3,"easy")

In [313]:
# with all
#precentage_2,correlation_coefficient_2,hit_precentage_2 = Evaluation(students_embed_asag,model_answer_embed_asag,df_val_final,"easy")

In [314]:
# with all 
#print(precentage_2,correlation_coefficient_2,hit_precentage_2)

it was 90.8569353667393 | 0.8572050623405271 | 0.4923747276688453


In [317]:
# without trans
print(precentage_2,correlation_coefficient_2,hit_precentage_2)

92.43697478991596 0.8156611109494591 0.5367647058823529


we concoulded that translated is bad for our model and asag is ever so slightly go up or down due its small size as well so we will use it if it make our model better note - we tried each one-.


now lets save it

In [318]:
final_model.save("/kaggle/working/Zingy_modeling")

In [319]:
# lets see if it saved

In [320]:
model = SentenceTransformer("/kaggle/working/Zingy_modeling")

In [321]:
model.encode(df_val["model_answer"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([-6.68196566e-03, -5.07425927e-02, -1.69533323e-02,  3.72392312e-02,
       -2.43939422e-02, -7.11960793e-02, -1.30633661e-03, -2.98047648e-03,
       -2.18982603e-02, -4.92835715e-02,  5.13281338e-02, -3.53322960e-02,
       -4.07093391e-02,  8.33557770e-02, -7.19670057e-02, -2.95926798e-02,
       -1.00199077e-02, -4.91048396e-03,  1.21934584e-03,  5.03767952e-02,
       -4.36901078e-02,  8.12573656e-02, -9.17903036e-02, -3.85606848e-02,
       -4.72458750e-02,  1.67687945e-02, -4.76532504e-02, -4.86967824e-02,
       -1.61977243e-02, -1.13298628e-03,  3.37589867e-02, -1.53830908e-02,
       -4.47862558e-02,  2.77485158e-02,  1.45667209e-03, -5.41925952e-02,
       -6.10473147e-03,  1.75780598e-02, -4.97622639e-02,  1.17753558e-02,
       -1.90768857e-02,  4.93073324e-03,  4.13859598e-02,  2.17588954e-02,
       -2.19567716e-02,  2.81837825e-02, -2.90717743e-02,  4.72225761e-03,
       -3.58409025e-02,  1.80729125e-02, -4.84117419e-02,  1.83361284e-02,
       -4.74200472e-02, -

In [322]:
!zip -r file.zip /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/Zingy_modeling/ (stored 0%)
  adding: kaggle/working/Zingy_modeling/modules.json (deflated 62%)
  adding: kaggle/working/Zingy_modeling/tokenizer.json (deflated 72%)
  adding: kaggle/working/Zingy_modeling/merges.txt (deflated 53%)
  adding: kaggle/working/Zingy_modeling/sentence_bert_config.json (deflated 4%)
  adding: kaggle/working/Zingy_modeling/1_Pooling/ (stored 0%)
  adding: kaggle/working/Zingy_modeling/1_Pooling/config.json (deflated 57%)
  adding: kaggle/working/Zingy_modeling/config_sentence_transformers.json (deflated 31%)
  adding: kaggle/working/Zingy_modeling/README.md (deflated 55%)
  adding: kaggle/working/Zingy_modeling/vocab.json (deflated 59%)
  adding: kaggle/working/Zingy_modeling/2_Normalize/ (stored 0%)
  adding: kaggle/working/Zingy_modeling/config.json (deflated 49%)
  adding: kaggle/working/Zingy_modeling/tokenizer_config.json (deflated 74%)

In [323]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Zingy_modeling	file.zip


In [324]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

now some exports 

In [261]:
#len(df_val)

In [262]:
#df_val.to_excel(r"/kaggle/working/Test_Dataset.xlsx",index=False)

In [263]:
#train_df_asag.to_excel(r"/kaggle/working/Traing_Data_614.xlsx",index=False)

In [264]:
#train_df_translated.to_excel(r"/kaggle/working/Traing_Data_translated.xlsx",index=False)

In [265]:
#train_df_og.to_excel(r"/kaggle/working/Traing_Data_analysis.xlsx",index=False)

In [328]:
train_og_final.to_excel(r"/kaggle/working/Traing_Data_analysis_&_mh.xlsx",index=False)


In [330]:
df_val_final_no3.to_excel(r"/kaggle/working/Val_data_set.xlsx",index=False)

In [332]:
train_df_asag.to_excel(r"/kaggle/working/asag_data_set.xlsx",index=False)